In [2]:
from __future__ import absolute_import, division, print_function

import os
import numpy as np
from collections import Counter


def load_velodyne_points(filename):
    """Load 3D point cloud from KITTI file format
    (adapted from https://github.com/hunse/kitti)
    """
    points = np.fromfile(filename, dtype=np.float32).reshape(-1, 4)
    points[:, 3] = 1.0  # homogeneous
    return points


def read_calib_file(path):
    """Read KITTI calibration file
    (from https://github.com/hunse/kitti)
    """
    float_chars = set("0123456789.e+- ")
    data = {}
    with open(path, 'r') as f:
        for line in f.readlines():
            key, value = line.split(':', 1)
            value = value.strip()
            data[key] = value
            if float_chars.issuperset(value):
                # try to cast to float array
                try:
                    data[key] = np.array(list(map(float, value.split(' '))))
                except ValueError:
                    # casting error: data[key] already eq. value, so pass
                    pass

    return data


def sub2ind(matrixSize, rowSub, colSub):
    """Convert row, col matrix subscripts to linear indices
    """
    m, n = matrixSize
    return rowSub * (n-1) + colSub - 1


def generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False):
    """Generate a depth map from velodyne data
    """
    # load calibration files
    cam2cam = read_calib_file(os.path.join(calib_dir, 'calib_cam_to_cam.txt'))
    velo2cam = read_calib_file(os.path.join(calib_dir, 'calib_velo_to_cam.txt'))
    velo2cam = np.hstack((velo2cam['R'].reshape(3, 3), velo2cam['T'][..., np.newaxis]))
    velo2cam = np.vstack((velo2cam, np.array([0, 0, 0, 1.0])))

    # get image shape
    im_shape = cam2cam["S_rect_02"][::-1].astype(np.int32)

    # compute projection matrix velodyne->image plane
    R_cam2rect = np.eye(4)
    R_cam2rect[:3, :3] = cam2cam['R_rect_00'].reshape(3, 3)
    P_rect = cam2cam['P_rect_0'+str(cam)].reshape(3, 4)
    P_velo2im = np.dot(np.dot(P_rect, R_cam2rect), velo2cam)

    # load velodyne points and remove all behind image plane (approximation)
    # each row of the velodyne data is forward, left, up, reflectance
    velo = load_velodyne_points(velo_filename)
    velo = velo[velo[:, 0] >= 0, :]

    # project the points to the camera
    velo_pts_im = np.dot(P_velo2im, velo.T).T
    velo_pts_im[:, :2] = velo_pts_im[:, :2] / velo_pts_im[:, 2][..., np.newaxis]

    if vel_depth:
        velo_pts_im[:, 2] = velo[:, 0]

    # check if in bounds
    # use minus 1 to get the exact same value as KITTI matlab code
    velo_pts_im[:, 0] = np.round(velo_pts_im[:, 0]) - 1
    velo_pts_im[:, 1] = np.round(velo_pts_im[:, 1]) - 1
    val_inds = (velo_pts_im[:, 0] >= 0) & (velo_pts_im[:, 1] >= 0)
    val_inds = val_inds & (velo_pts_im[:, 0] < im_shape[1]) & (velo_pts_im[:, 1] < im_shape[0])
    velo_pts_im = velo_pts_im[val_inds, :]

    # project to image
    depth = np.zeros((im_shape[:2]))
    depth[velo_pts_im[:, 1].astype(np.int), velo_pts_im[:, 0].astype(np.int)] = velo_pts_im[:, 2]

    # find the duplicate points and choose the closest depth
    inds = sub2ind(depth.shape, velo_pts_im[:, 1], velo_pts_im[:, 0])
    dupe_inds = [item for item, count in Counter(inds).items() if count > 1]
    for dd in dupe_inds:
        pts = np.where(inds == dd)[0]
        x_loc = int(velo_pts_im[pts[0], 0])
        y_loc = int(velo_pts_im[pts[0], 1])
        depth[y_loc, x_loc] = velo_pts_im[pts, 2].min()
    depth[depth < 0] = 0

    return depth


In [3]:
calib_dir = "/home/hanzhic/KITTI_Depth_Demo/calib/"
velo_filename = "/home/hanzhic/KITTI_Depth_Demo/2011_09_26/2011_09_26_drive_0001_sync/velodyne_points/data/0000000000.bin"

generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False)
N = 0
test_depth = generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False)
print(np.max(test_depth))
for i in range(375):
    for j in range(1242):
        if test_depth[i][j] != 0.0:
            N+=1
            print((i,j), test_depth[i][j])
print("number of anchors", N)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 6.32407174, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 5.40322175, 0.        ,
        0.        ]])

In [6]:
N = 0
test_depth = generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False)
print(np.max(test_depth))
for i in range(375):
    for j in range(1242):
        if test_depth[i][j] != 0.0:
            N+=1
            print((i,j), test_depth[i][j])
print("number of anchors", N)


78.39623801504047
(129, 1226) 18.166370845962806
(129, 1230) 17.913304644122523
(129, 1234) 18.175397769772005
(129, 1236) 18.15039590999437
(129, 1240) 17.972348110144306
(130, 1210) 18.904555754342663
(130, 1214) 18.612474330452027
(130, 1218) 18.47445188985239
(130, 1222) 18.3904322101391
(131, 1189) 19.557701498086118
(131, 1193) 19.470681349693496
(131, 1196) 19.52771141814345
(131, 1200) 19.25463767937473
(131, 1204) 19.192623871843928
(131, 1208) 19.13062064052018
(132, 1172) 20.273869165072004
(132, 1176) 20.157843182918583
(132, 1179) 20.088830161312156
(132, 1183) 19.947796304062237
(132, 1185) 19.869781790061207
(133, 1046) 16.164438213830795
(133, 1050) 16.145443265276725
(133, 1051) 16.156447696276217
(133, 1054) 16.024414249471825
(133, 1058) 15.923387584708342
(133, 1061) 15.958405432618012
(133, 1151) 21.32912889024124
(133, 1155) 21.178095224693415
(133, 1159) 21.004045342771388
(133, 1161) 20.904023836075392
(133, 1164) 20.788994969651263
(133, 1168) 20.63996225351083

(161, 814) 44.555165500197994
(161, 817) 44.47815387344033
(161, 819) 45.106180904537496
(161, 822) 45.06716759978469
(161, 859) 55.70973742788988
(161, 862) 55.0106756212162
(161, 863) 54.337628503080545
(161, 866) 53.27053076533516
(161, 868) 52.199446890060685
(161, 871) 52.16242495639217
(161, 873) 51.33836122094764
(161, 876) 50.798307926161115
(161, 879) 49.922238356748394
(161, 880) 49.44520100155344
(161, 882) 48.74813607567929
(161, 996) 31.362746197168835
(161, 1021) 18.32516552356104
(161, 1024) 18.294159741634797
(161, 1026) 18.281156107985097
(161, 1029) 18.270148199173786
(161, 1154) 21.51845864048616
(161, 1156) 21.374472213344852
(161, 1160) 21.219483266733512
(161, 1163) 21.06848108023396
(161, 1167) 20.903493694549912
(161, 1171) 20.75949072205172
(161, 1174) 20.603500969519214
(161, 1178) 20.46650956982103
(161, 1180) 20.363507585011977
(162, 0) 20.740170674919156
(162, 4) 20.739144991542435
(162, 8) 20.71010955430982
(162, 12) 20.698083797225074
(162, 15) 20.6910594

(188, 163) 26.136793847271573
(188, 166) 26.23977016185065
(188, 169) 26.340757886955465
(188, 172) 26.568717583057847
(188, 176) 26.605704776792372
(188, 179) 26.683680723988797
(188, 180) 26.75867577433669
(188, 183) 24.531006534027043
(188, 256) 30.71389167255864
(188, 259) 30.767877887539413
(188, 261) 30.866862425830956
(188, 263) 32.11166132741868
(188, 266) 32.09165574801971
(188, 268) 32.066652939827335
(188, 301) 33.85826913787455
(188, 304) 34.09322207847265
(188, 307) 34.316173890610216
(188, 309) 34.36315877739787
(188, 312) 34.59410982136148
(188, 315) 34.72008248807011
(188, 316) 36.056861569159146
(188, 319) 36.232822032337374
(188, 321) 36.44878309174732
(188, 324) 36.773723678789935
(188, 327) 36.92168392743582
(188, 333) 35.62187181487325
(188, 336) 35.58286502281436
(188, 409) 46.93562825866751
(188, 412) 46.93062170677941
(188, 414) 46.96259886082915
(188, 417) 47.014579941134514
(188, 418) 46.927580657761276
(188, 461) 60.52090210772998
(188, 463) 59.49006633478308

(208, 397) 26.06650374696022
(208, 400) 26.2843948333877
(208, 402) 26.735166117277636
(208, 405) 26.99003390014651
(208, 421) 31.236900365319453
(208, 423) 31.24889648247503
(208, 426) 31.22790481863525
(208, 428) 31.19291360731833
(208, 429) 31.270874911724402
(208, 437) 31.288866254297254
(208, 445) 31.728642456881836
(208, 448) 31.979507018415372
(208, 450) 32.27835996044492
(208, 460) 34.34130567777835
(208, 461) 34.390276039459486
(208, 463) 34.563183943689296
(208, 466) 34.8320478336003
(208, 481) 38.68507356812869
(208, 483) 38.57712243255199
(208, 486) 38.46217539990384
(208, 488) 38.410190974564465
(208, 490) 38.34121939083514
(208, 493) 38.284248436835675
(208, 494) 38.231276390529324
(208, 496) 38.15130369640135
(208, 499) 37.99237718615477
(208, 501) 37.86843903324185
(208, 503) 37.807460293001014
(208, 506) 37.70649941210352
(208, 508) 37.59854927019114
(208, 509) 37.492598870757774
(208, 511) 37.41163134115477
(208, 514) 37.30868224295482
(208, 516) 37.25270110506588
(20

(225, 295) 27.978802312782197
(225, 297) 27.986795202211404
(225, 300) 28.042771375661637
(225, 302) 28.043789066338366
(225, 421) 25.434037579028615
(225, 423) 25.391078945466113
(225, 426) 25.317143682207718
(225, 428) 25.273175201997567
(225, 431) 25.23520728477179
(225, 432) 25.183255495183378
(225, 434) 25.13828774198155
(225, 437) 25.097332088240695
(225, 439) 25.089330954063158
(225, 442) 25.057363476651975
(225, 444) 25.009406329649217
(225, 446) 24.961438605312118
(225, 448) 24.902487228519682
(225, 450) 24.84354132884894
(225, 538) 22.409198830255797
(225, 539) 22.41518421676736
(225, 541) 22.402196137745406
(225, 544) 22.41917625159071
(225, 546) 22.44315595139414
(225, 548) 22.449147312556633
(225, 551) 22.44713765879729
(225, 553) 22.467117743822335
(225, 554) 22.46212292221283
(225, 556) 22.47111437892626
(225, 559) 22.46511568530521
(225, 561) 22.47809618188034
(225, 563) 22.468097872235017
(225, 566) 22.47708932894844
(225, 568) 22.459092034765874
(225, 569) 22.47707589

...

(275, 687) 11.395098104462711
(275, 689) 11.376115638229974
(275, 691) 11.378113021107687
(275, 693) 11.382097335086554
(275, 695) 11.378093366278184
(275, 698) 11.376078568394497
(275, 700) 11.393041633971185
(275, 723) 11.126372185235011
(275, 724) 10.930697697960683
(275, 726) 10.759977169480772
(275, 729) 10.696072317424523
(275, 731) 10.651133924675744
(275, 734) 10.521337742674667
(275, 736) 10.445453908221495
(275, 738) 10.455428211484293
(275, 740) 10.536283315622905
(275, 742) 10.53027984973037
(275, 744) 10.55921086540393
(275, 747) 10.51627234283806
(275, 749) 10.578158873477577
(275, 752) 10.545198881341511
(275, 754) 10.616063593961988
(275, 755) 10.504247414262808
(275, 760) 10.70985689253204
(275, 811) 10.021702406134969
(275, 921) 10.876038744553354
(275, 924) 10.744246298955975
(275, 927) 10.73922070498759
(275, 989) 10.546442591293797
(275, 991) 10.512485359810727
(275, 994) 10.541403685100436
(275, 997) 10.54136698453858
(275, 1000) 10.48044223629233
(275, 1003) 10.5

(297, 81) 11.084344063330237
(297, 113) 10.844504499813894
(297, 117) 10.846529381535365
(297, 120) 10.86553412321461
(297, 174) 10.486832625840155
(297, 178) 10.490847672091334
(297, 181) 10.507852915470174
(297, 182) 10.505871088436447
(297, 185) 10.517876366081442
(297, 283) 9.764659391889166
(297, 284) 9.737720245710186
(297, 287) 9.740736766472367
(297, 290) 9.739763044978579
(297, 292) 9.74476886102181
(297, 395) 9.566640601527466
(297, 398) 9.60059435201136
(297, 399) 9.599602338596362
(297, 402) 9.61058818957242
(297, 404) 9.607604643589099
(297, 407) 9.623589505430873
(297, 409) 9.628595818899562
(297, 412) 9.637591296615971
(297, 549) 9.479307770461533
(297, 551) 9.475314875270843
(297, 554) 9.482310607153297
(297, 556) 9.49428540118682
(297, 558) 9.487302862236586
(297, 559) 9.492290386343782
(297, 562) 9.502274932727325
(297, 564) 9.506261072128172
(297, 566) 9.50626766734468
(297, 569) 9.500274072712628
(297, 571) 9.498280923348485
(297, 574) 9.506266553156529
(297, 575) 9

(322, 1201) 6.9966973716052205
(322, 1203) 6.958773866874808
(322, 1207) 6.9577277121028445
(323, 0) 8.924536239199464
(323, 4) 8.94254112739319
(323, 80) 8.635873409785336
(323, 83) 8.649878931928232
(323, 85) 8.667855699437075
(323, 150) 8.344252571755547
(323, 151) 8.362230417302289
(323, 155) 8.36625678595801
(323, 158) 8.37925295763282
(323, 211) 8.170444855736935
(323, 214) 8.16748304135636
(323, 217) 8.17548874546648
(323, 286) 8.034517102566525
(323, 288) 8.047502367588018
(323, 291) 8.049520238415884
(323, 293) 8.066496533119514
(323, 295) 8.066513580112584
(323, 398) 7.878521076493815
(323, 400) 7.885517554581169
(323, 403) 7.8935142580068955
(323, 406) 7.8855419442489545
(323, 407) 7.8895398191507
(323, 409) 7.9075052243018655
(323, 412) 7.912512283968044
(323, 545) 7.94101466791069
(323, 548) 7.940012038305852
(323, 550) 7.927050706807256
(323, 553) 7.951994672461788
(323, 554) 7.950992705986056
(323, 556) 7.94401066325245
(323, 558) 7.948006921410266
(323, 561) 7.948003687

(346, 952) 6.660718631382747
(346, 955) 6.642738153530451
(346, 958) 6.655670767068523
(346, 961) 6.652647136845507
(346, 964) 6.6306883171734885
(346, 1034) 6.665530515776062
(346, 1037) 6.653528351146987
(346, 1040) 6.649504122486851
(346, 1043) 6.652458681160343
(346, 1045) 6.614545255092922
(346, 1122) 5.98412536331928
(346, 1124) 5.958179514220336
(346, 1128) 5.903298649192619
(346, 1131) 5.878329780849631
(346, 1205) 5.911134780475277
(346, 1209) 5.88217724415309
(347, 54) 7.289044932321261
(347, 57) 7.308040610862077
(347, 59) 7.320028468385609
(347, 136) 7.011508349768362
(347, 138) 7.013517283089704
(347, 141) 7.02452370459188
(347, 203) 6.909572926101518
(347, 206) 6.93054854678424
(347, 209) 6.941545242048529
(347, 267) 6.910443435999629
(347, 270) 6.924429546954729
(347, 272) 6.931427890506153
(347, 275) 6.934435204345788
(347, 338) 6.8524016377314885
(347, 341) 6.8683882213125225
(347, 343) 6.8763848210258764
(347, 345) 6.862425212699121
(347, 347) 6.8863797985271615
(347,

(372, 395) 5.998182591410337
(372, 397) 5.9962010118730955
(372, 400) 6.008177299554902
(372, 402) 6.008195465844205
(372, 405) 6.0231610448308075
(372, 485) 5.902270524228996
(372, 487) 5.915236233017865
(372, 490) 5.911254555214151
(372, 492) 5.927220483925142
(372, 494) 5.921239061540778
(372, 497) 5.930225704209731
(372, 498) 5.933213627505563
(372, 501) 5.931232297949181
(372, 795) 5.893767923085506
(372, 797) 5.919679080811479
(372, 800) 5.920655916522604
(372, 802) 5.920632402509339
(372, 805) 5.916630071111127
(372, 806) 5.9156289545304315
(372, 808) 5.905637483758152
(372, 811) 5.9026353776983145
(372, 908) 5.549556079042499
(372, 911) 5.656197506163411
(372, 914) 5.596368309996218
(372, 917) 5.501636980025869
(372, 920) 5.485667071644355
(372, 1000) 5.64611199064626
(372, 1003) 5.643088256710873
(372, 1006) 5.64604343847138
(372, 1076) 5.7406905679593745
(372, 1079) 5.697787232960867
(372, 1083) 5.655883170924392
(372, 1158) 5.593965919305919
(372, 1162) 5.559030442829606
(37